# Time Series test

## Import Libraries

In [2]:

import sys
import os

import pandas as pd
import numpy as np

import sympy as sym
sym.init_printing()
from IPython.display import display, Math

from statsmodels.tsa.stattools import adfuller
from scipy.stats import boxcox
import plotly.express as px
import plotly.graph_objects as go


sys.path.append(os.path.abspath('..'))
from utils import find_missing_dates
from utils.fill_dates import fill_missing_dates
from utils.diff import custom_diff
from utils.log import custom_ln
from utils.acf import custom_acf

#### SQL Query

select sale_date, concat(product_code, ' - ', product_description) as product, sum(total_sales) as sales
from sales
where sale_date between '1/1/2025' and '6/1/2025'
and store_number='440'
and sale_type in ('Refunded', 'Sale')
and item_ring_type in ('ITEM', 'SUBD')
and category = 90201
and product_code='28000600000.0'
group by sale_date, product_code, product_description
order by sale_date

## Import Data


In [3]:

ds = pd.read_csv('../data/store-440-fresh_ground_chuck.csv')

In [24]:
#ds.info()

In [22]:
#ds.describe()

In [3]:
ds.head()

,sale_date,product,sales
0,2025-02-24 00:00:00,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,78.96
1,2025-02-25 00:00:00,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,83.67
2,2025-02-26 00:00:00,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,28.13
3,2025-02-27 00:00:00,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,54.29
4,2025-02-28 00:00:00,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,57.99


## check data

In [4]:
startdate = ds['sale_date'].min()
enddate = ds['sale_date'].max()
missing = find_missing_dates(ds, "sale_date", startdate, enddate)
print(missing)
# This will occur if this item does not have sales for this day

[Timestamp('2025-04-11 00:00:00')]


In [5]:
# fill the missing dates
ds = fill_missing_dates(ds, "sale_date", 'sales', startdate, enddate, 0.01)

In [5]:
# check for values that are less than or equal to zero
ds[ds['sales'] <= 0]

,sale_date,product,sales


## Plot our data

In [6]:
# we are going to add the day of the week to make things easier to look at
ds['day_of_week'] = pd.to_datetime(ds['sale_date']).dt.day_name()

In [7]:
def plot(title, data, x, y, x_label, y_label):
    """ General function to plot data"""
    fig = px.line(
        data, 
        x=data[x], 
        y=data[y], 
        title='Store 440 Fresh Ground Chuck Sales',
        labels={x: x_label, y: y_label},
        hover_data={'sale_date': True, 'sales': True, 'day_of_week': True}
        )

    fig.update_layout(
        template='simple_white',
        font=dict(size=16),
        width=1400,
        height=450,
        title_x=0.5,
        xaxis=dict(
            tickangle=-45,
            tickformat="%b %d",
            showgrid=True,
            rangeselector=dict(
                buttons=list([
                    dict(count=7, label="1w", step="day", stepmode="backward"),
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        ),
        yaxis=dict(showgrid=True)
    )
    
    fig.show()

In [8]:
plot(title='Fresh Ground Chuck Price', data=ds, x='sale_date', y='sales',
     x_label='Date', y_label='Sales')

i:\Code_maths\env\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


<img src="../images/calendar-mar-jun_v1.png" alt="calendar" style="width:33%; display:block; margin:0 auto;" />

## Make the Time Series Stationary

#### run a differencing function

$$
    d(t) = y(t) - y(t-1)
$$

![diff](../images/diff.png)

In [9]:
# get the difference between y(t) and y(t-1)
ds['diff'] = ds['sales'].diff()
ds['custom_diff'] = custom_diff(ds['sales'].tolist())


In [10]:
ds.head()

,sale_date,product,sales,day_of_week,diff,custom_diff
0,2025-02-24,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,78.96,Monday,NaN,NaN
1,2025-02-25,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,83.67,Tuesday,4.71,4.71
2,2025-02-26,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,28.13,Wednesday,-55.54,-55.54
3,2025-02-27,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,54.29,Thursday,26.16,26.16
4,2025-02-28,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,57.99,Friday,3.70,3.70


In [10]:
# 🤘 this time we want to plot the difference. of course, we are going to use our implementation because it rocks!!
plot(title='Fresh Ground Chuck Price', data=ds, x='sale_date', y='custom_diff',
     x_label='Date', y_label='Sales')

i:\Code_maths\env\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



<img src="../images/calendar-mar-jun_v1.png" alt="calendar" style="width:33%; display:block; margin:0 auto;" />

In [12]:
ds.head()

,sale_date,product,sales,day_of_week,diff,custom_diff
0,2025-02-24,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,78.96,Monday,NaN,NaN
1,2025-02-25,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,83.67,Tuesday,4.71,4.71
2,2025-02-26,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,28.13,Wednesday,-55.54,-55.54
3,2025-02-27,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,54.29,Thursday,26.16,26.16
4,2025-02-28,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,57.99,Friday,3.70,3.70


#### Logarithm Transform

In [11]:
ds['log'] = np.log(ds['sales'])

In [12]:
ds.head()

,sale_date,product,sales,day_of_week,diff,custom_diff,log
0,2025-02-24,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,78.96,Monday,NaN,NaN,4.368941
1,2025-02-25,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,83.67,Tuesday,4.71,4.71,4.426880
2,2025-02-26,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,28.13,Wednesday,-55.54,-55.54,3.336837
3,2025-02-27,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,54.29,Thursday,26.16,26.16,3.994340
4,2025-02-28,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,57.99,Friday,3.70,3.70,4.060271


Lets see how well our custom log function holds up

In [13]:
ds['custom_log'] = [custom_ln(x) for x in  ds['sales']]

In [14]:
ds.head()

,sale_date,product,sales,day_of_week,diff,custom_diff,log,custom_log
0,2025-02-24,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,78.96,Monday,NaN,NaN,4.368941,4.367879
1,2025-02-25,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,83.67,Tuesday,4.71,4.71,4.426880,4.425395
2,2025-02-26,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,28.13,Wednesday,-55.54,-55.54,3.336837,3.336837
3,2025-02-27,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,54.29,Thursday,26.16,26.16,3.994340,3.994264
4,2025-02-28,28000600000.0 - FRESH GROUND CHUCK 80/20 LB,57.99,Friday,3.70,3.70,4.060271,4.060141


In [15]:
# we are going to plot the log and see what it looks like
plot(title='Fresh Ground Chuck Price', data=ds, x='sale_date', y='custom_log',
     x_label='Date', y_label='Sales')

i:\Code_maths\env\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## check for stationarity

#### Run the adfuller test

In [16]:
def adf_test(series):
    """Using an ADF test to determine if a series is stationary"""
    test_results = adfuller(series)
    print('ADF Statistic: ', test_results[0])
    print('P-Value: ', test_results[1])
    print('Critical Values: ')
    for thres, adf_stat in test_results[4].items():
        print('\t%s: %.2f' % (thres, adf_stat))

In [17]:
adf_test(ds["custom_log"][1:])

ADF Statistic:  -9.59883140604306
P-Value:  1.9452373054541786e-16
Critical Values: 
	1%: -3.50
	5%: -2.89
	10%: -2.58


#### Plot all the steps

In [18]:
def plot_v2(title, data, x, y, x_label, y_label):
    """ General function to plot data"""
    fig = px.line(
        data, 
        x=data[x], 
        y=data[y], 
        title='Store 440 Fresh Ground Chuck Sales',
        labels={x: x_label, y: y_label},
        hover_data={'sale_date': True, 'sales': True, 'day_of_week': True}
        )
    
    fig.add_trace(
        go.Scatter(
            x = data[x],
            y = data['custom_diff'],
            mode='lines',
            name='Difference',
            line = dict(dash='dashdot') # solid | dash | dot | dashdot
        )
    )

    fig.add_trace(
        go.Scatter(
            x = data[x],
            y = data['custom_log'],
            mode='lines',
            name='Logorithmic',
            line = dict(dash='dash') # solid | dash | dot | dashdot
        )
    )


    fig.update_layout(
        template='simple_white',
        font=dict(size=16),
        width=1400,
        height=450,
        title_x=0.5,
        xaxis=dict(
            tickangle=-45,
            tickformat="%b %d",
            showgrid=True,
            rangeselector=dict(
                buttons=list([
                    dict(count=7, label="1w", step="day", stepmode="backward"),
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        ),
        yaxis=dict(showgrid=True)
    )
    
    fig.show()

In [19]:
# we are going to plot the log and see what it looks like
plot_v2(title='Fresh Ground Chuck Price', data=ds, x='sale_date', y='sales',
     x_label='Date', y_label='Sales')

i:\Code_maths\env\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

i:\Code_maths\env\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## Correlations

### ACF
Auto Correlation Function

In [20]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import acf

In [21]:
nlags = 30
ts = ds['sales']
acf_values, confint = acf(ts, nlags=nlags, alpha=0.05)

- acf_values is a NumPy array of length nlags+1, where acf_values[0] == 1.0
- confint is an array of shape (nlags+1, 2) giving the 95% confidence interval for each lag

In [43]:
#confint

In [22]:
# lets bring our custom_acf function and compare the differences
my_acf, ci_lo, ci_hi = custom_acf(ts, nlags=nlags)

In [23]:
len(my_acf)

In [53]:
# lets see if we match or not
# for k in range(nlags+1):
#     print(f"{k:>3} | {my_acf[k]:> .4f}    |    {acf_values[k]:> .4f}")

In [24]:
lags = list(range(len(acf_values)))
acf_ds = pd.DataFrame({
    'lag': lags,
    'acf': acf_values
})

In [27]:
acf_ds.head()

,lag,acf
0,0,1.000000
1,1,0.101428
2,2,-0.037024
3,3,-0.096887
4,4,-0.080185


In [25]:
def plot_acf(ts, nlags=30, alpha=0.5):
    acf_vals, confint = acf(ts, nlags=nlags, alpha=alpha, fft=True)
    print(f'acf_vals: {acf_vals}')
    print(f'confint: {confint}')
    lags = list(range(len(acf_vals)))

    acf_ds = pd.DataFrame({'lag': lags, 'acf': acf_vals})

    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            x=acf_ds['lag'],
            y=acf_ds['acf'],
            marker_color='steelblue',
            width=0.2,
            name='ACF'
        )
    )

    fig.add_trace(
        go.Scatter(
            x=acf_ds['lag'],
            y=acf_ds['acf'],
            mode='markers',
            marker=dict(color='crimson', size=8),
            name='ACF points'
        )
    )

    lower_ci = confint[:, 0]
    upper_ci = confint[:, 1]
    print(f'upper_ci: {upper_ci}')
    print(f'lower_ci: {lower_ci}')
    fig.add_trace(
        go.Scatter(
            x=acf_ds["lag"].tolist() + acf_ds["lag"].tolist()[::-1],
            y=(lower_ci.tolist() + upper_ci.tolist()[::-1]),
            fill="toself",
            fillcolor="rgba(0, 0, 200, 0.1)",
            line=dict(color="rgba(255,255,255,0)"),
            hoverinfo="skip",
            showlegend=True,
            name=f"{int((1 - alpha)*100)}% CI"
        )
    )  
    fig.update_layout(
        title=f"Autocorrelation Function (ACF) — nlags={nlags}",
        template="simple_white",
        width=800,
        height=450,
        xaxis=dict(
            tickmode="linear",
            tick0=0,
            dtick=1,
            title="Lag"
        ),
        yaxis=dict(
            range=[-1.0, 1.0],
            title="ACF"
        )
    )

    fig.show()      

In [26]:
plot_acf(ts, nlags=25, alpha=0.05)

acf_vals: [ 1.          0.10142841 -0.03702448 -0.09688727 -0.08018476 -0.02024966
 -0.18657538 -0.09514073 -0.01834926  0.06728032 -0.10954052  0.06743544
  0.02429182 -0.02524808 -0.1038728  -0.16709635 -0.06465681  0.03124796
  0.16610807  0.01609843 -0.00259081  0.01953366  0.06678378 -0.01083312
  0.06000351 -0.03417754]
confint: [[ 1.          1.        ]
 [-0.09655785  0.29941467]
 [-0.2370372   0.16298824]
 [-0.29716846  0.10339392]
 [-0.28229483  0.12192531]
 [-0.2236029   0.18310359]
 [-0.39000764  0.01685689]
 [-0.3051734   0.11489194]
 [-0.23006452  0.19336601]
 [-0.14449728  0.27905791]
 [-0.32215432  0.10307327]
 [-0.14737918  0.28225006]
 [-0.19135102  0.23993466]
 [-0.24099816  0.190502  ]
 [-0.31973867  0.11199307]
 [-0.38491266  0.05071995]
 [-0.2874412   0.15812757]
 [-0.19227077  0.25476669]
 [-0.05758183  0.38979798]
 [-0.21237541  0.24457228]
 [-0.23110911  0.22592749]
 [-0.20898579  0.24805312]
 [-0.16180112  0.29536867]
 [-0.24018157  0.21851532]
 [-0.169365    

In [27]:
# now lets plot this using our function
def plot_acf_custom(ts, nlags=30, alpha=0.05):
    acf_vals, ci_lo, ci_hi = custom_acf(ts, nlags=nlags)
    lags = list(range(len(acf_vals)))
    print(f"ci_hi: {ci_hi}")
    print(f'ci_lo: {ci_lo}')

    acf_ds = pd.DataFrame({
        'lag': lags, 
        'acf': acf_vals,
        'ci_lo': ci_lo,
        'ci_hi': ci_hi        
        })

    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            x=acf_ds['lag'],
            y=acf_ds['acf'],
            marker_color='steelblue',
            width=0.2,
            name='ACF'
        )
    )

    fig.add_trace(
        go.Scatter(
            x=acf_ds['lag'],
            y=acf_ds['acf'],
            mode='markers',
            marker=dict(color='crimson', size=8),
            name='ACF points'
        )
    )

    fig.add_trace(
        go.Scatter(
            x=acf_ds["lag"].tolist() + acf_ds["lag"].tolist()[::-1],
            y=(acf_ds['ci_lo'].tolist() + acf_ds['ci_hi'].tolist()[::-1]),
            fill="toself",
            fillcolor="rgba(0, 0, 200, 0.1)",
            line=dict(color="rgba(255,255,255,0)"),
            hoverinfo="skip",
            showlegend=True,
            name=f"95% CI"
        )
    )  
    fig.update_layout(
        title=f"Autocorrelation Function (ACF) — nlags={nlags}",
        template="simple_white",
        width=800,
        height=450,
        xaxis=dict(
            tickmode="linear",
            tick0=0,
            dtick=1,
            title="Lag"
        ),
        yaxis=dict(
            range=[-1.0, 1.0],
            title="ACF"
        )
    )

    fig.show()      

In [28]:
plot_acf_custom(ts, nlags=25)

ci_hi: [1.         0.29941831 0.16299192 0.1033976  0.12192903 0.18310732
 0.01686063 0.1148958  0.1933699  0.2790618  0.10307717 0.28225401
 0.23993863 0.19050596 0.11199703 0.05072395 0.15813167 0.2547708
 0.38980209 0.24457648 0.22593169 0.24805732 0.29537287 0.21851954
 0.28937623 0.19580967]
ci_lo: [ 1.         -0.09656149 -0.23704087 -0.29717214 -0.28229854 -0.22360664
 -0.39001138 -0.30517725 -0.23006841 -0.14450117 -0.32215822 -0.14738313
 -0.19135498 -0.24100213 -0.31974263 -0.38491666 -0.2874453  -0.19227488
 -0.05758594 -0.21237961 -0.23111331 -0.20898999 -0.16180532 -0.24018579
 -0.16936922 -0.26416476]
